In [1]:
from src import config
from src.datasets import TextConcatFactCheck, TextConcatPosts

# Paths from config
posts_path = config.POSTS_PATH
fact_checks_path = config.FACT_CHECKS_PATH
gs_path = config.GS_PATH
task_path = config.TASKS_PATH
task_name = "monolingual"
lang = "tha"

posts = TextConcatPosts(posts_path, tasks_path=task_path, gs_path=gs_path, task_name=task_name, lang=lang)
fcs = TextConcatFactCheck(fact_checks_path, tasks_path=task_path, task_name=task_name, lang=lang)

# df_posts_train = posts.df_train
df_posts_dev = posts.df_dev
df_fc = fcs.df

In [2]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import torch
import numpy as np
from abc import abstractmethod
import json
import os
from src.models import BaseModel

class CrossencoderModel(BaseModel):
    def __init__(self, model_name, df_cands, df_fc, device="cuda", show_progress_bar=True, batch_size=128, k=10, **kwargs):
        self.model = CrossEncoder(model_name, device=device, **kwargs)
        self.idx_to_text = df_fc["full_text"].to_dict()
        self.df_cands = df_cands.copy()
        self.vectorized_map = np.vectorize(lambda x: self.idx_to_text.get(x, None))        
        super().__init__(device, show_progress_bar, batch_size, k)

    def train(self, texts):
        pass
    
    def predict(self, post, cands_list):
        # df_posts_dev["preds"] = df_posts_dev.apply(lambda x:cross_model.predict(x["full_text"], x["preds"]), axis=1)
        cands_list_text = self.vectorized_map(cands_list)
        pos_ids = self.model.rank(post, cands_list_text, show_progress_bar=self.show_progress_bar, batch_size=self.batch_size, top_k=self.k, convert_to_numpy=True)
        pos_ids = [pos_id["corpus_id"] for pos_id in pos_ids]
        return np.array(cands_list)[pos_ids]

/home/bsc/bsc830651/.conda/envs/factcheck/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
from src.models import EmbeddingModel

biencoder_name = "/gpfs/projects/bsc14/abecerr1/hub/models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2/snapshots/ae06c001a2546bef168b9bf8f570ccb1a16aaa27"
cand_ret_model = EmbeddingModel(biencoder_name, df_fc, show_progress_bar=True, batch_size=256, normalize_embeddings=True, k=100)

df_posts_dev["preds"] = cand_ret_model.predict(df_posts_dev["full_text"].values).tolist()



Batches: 100%|██████████| 1/1 [00:00<00:00, 30.47it/s]


In [11]:
# cross_model = CrossencoderModel("amberoad/bert-multilingual-passage-reranking-msmarco", df_posts_dev, df_fc, show_progress_bar=False, batch_size=512, k=10)

cross_base = CrossEncoder("amberoad/bert-multilingual-passage-reranking-msmarco", device="cuda")
cross_base.predict([("Hola", "Hallo")])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


array([[ 2.8971457, -2.334918 ]], dtype=float32)

In [ ]:
softmax = torch.nn.Softmax(dim=0)

In [14]:
model(**tokenizer(["Hola", "Hallo"], return_tensors="pt")).logits

tensor([[ 2.5404, -2.0186],
        [ 2.4510, -1.9740]], grad_fn=<AddmmBackward0>)

In [4]:
cross_model = CrossencoderModel("amberoad/bert-multilingual-passage-reranking-msmarco", df_posts_dev, df_fc, show_progress_bar=False, batch_size=512, k=10)

df_posts_dev["preds_cross"] = df_posts_dev.apply(lambda x:cross_model.predict(x["full_text"], x["preds"]), axis=1)

df_posts_dev_2 = df_posts_dev.copy()
df_posts_dev_2["preds"] = df_posts_dev_2["preds_cross"]

cross_model.evaluate(df_posts_dev_2, task_name=task_name, lang=lang)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
cross_model.evaluate(df_posts_dev_2, task_name="monolingual", lang="fra")

{'monolingual': {'fra': {1: np.float64(0.46405228758169936),
   3: np.float64(0.6013071895424836),
   5: np.float64(0.6470588235294118),
   10: np.float64(0.6993464052287581)}}}

In [4]:
df_fc.iloc[:10]["full_text"].to_dict()

{15: 'whatsapp : une vidéo « martinelli » qui va contaminer votre téléphone ? aucune preuve en ce sens " il y a une vidéo qui sera lancée demain à whatsapp et qui s’appelle martinelli. "',
 136: 'fatou sarr sow se trompe sur le taux d’alphabétisation des femmes au sénégal "1960 on n’était que 3% des femmes alphabétisées. si on regarde aujourd’hui on n’est pas satisfait parce qu’on a 46-47%"',
 155: 'est-il vrai que 25% des effets secondaires des vaccins contre le covid sont «graves» ? "25% des effets secondaires liés aux vaccins sont graves"',
 156: 'non, les autorités congolaises n’ont pas enregistré la reddition de "26 groupes armés" dans l’est de la rdc "26 groupes armés" ont rendu les armes dans la province de l\'ituri en république démocratique du congo',
 187: 'sénégal : 5 % des élèves consomment-ils de la drogue ? "5 % des élèves consomment de la drogue".',
 214: '"80% des blancs sont tués par des noirs" aux etats-unis, selon zemmour. c\'est cinq fois moins "80% des blancs sont 

In [6]:
idx_to_text = df_fc["full_text"].to_dict()

fun = np.vectorize(lambda x: idx_to_text.get(x, None))

df_posts_dev["preds_text"] = df_posts_dev["preds"].apply(fun)

In [7]:
ce_model = CrossEncoder("cross-encoder/stsb-distilroberta-base", device="cuda")

In [8]:
import torch
sigmoid = torch.nn.Sigmoid()
scores = df_posts_dev.apply(lambda x: ce_model.rank(x["full_text"], x["preds_text"], top_k=10, convert_to_numpy=True), axis=1)
# model = CrossEncoder('model_name', max_length=512)
# scores = model.predict([('Query', 'Paragraph1'), ('Query', 'Paragraph2') , ('Query', 'Paragraph3')])
scores

post_id
291      [{'corpus_id': 0, 'score': 0.81704694}, {'corp...
675      [{'corpus_id': 73, 'score': 0.61601925}, {'cor...
1073     [{'corpus_id': 88, 'score': 0.49774748}, {'cor...
1292     [{'corpus_id': 36, 'score': 0.5635742}, {'corp...
1299     [{'corpus_id': 68, 'score': 0.6377366}, {'corp...
                               ...                        
27843    [{'corpus_id': 93, 'score': 0.8546004}, {'corp...
27856    [{'corpus_id': 75, 'score': 0.63924944}, {'cor...
27882    [{'corpus_id': 0, 'score': 0.5952376}, {'corpu...
27896    [{'corpus_id': 2, 'score': 0.589157}, {'corpus...
27897    [{'corpus_id': 13, 'score': 0.5853241}, {'corp...
Length: 153, dtype: object

In [15]:
pos_preds = scores.map(lambda x: [i["corpus_id"] for i in x])
df_posts_dev["pos_preds"] = pos_preds

In [16]:
df_posts_dev.apply(lambda x: np.array(x["preds"])[np.array(x["pos_preds"])], axis=1)

post_id
291      [10453, 82708, 79322, 155790, 11378, 80408, 15...
675      [36592, 83261, 80178, 27977, 27772, 147115, 44...
1073     [22026, 28270, 137418, 36663, 44372, 82576, 82...
1292     [23311, 78538, 136865, 138187, 61810, 61671, 5...
1299     [27774, 22026, 61671, 138129, 3284, 83546, 522...
                               ...                        
27843    [92820, 137024, 36592, 150822, 79482, 150823, ...
27856    [74919, 27774, 9319, 40405, 138113, 36548, 831...
27882    [82722, 36660, 92696, 87120, 81432, 52283, 400...
27896    [22026, 155810, 44367, 92820, 156184, 36660, 7...
27897    [44252, 36550, 138129, 137990, 3284, 36553, 11...
Length: 153, dtype: object

In [23]:
print(df_posts_dev["full_text"].iloc[0])
idxs = df_posts_dev["pos_preds"].iloc[0]
df_posts_dev["preds_text"].iloc[0][idxs]

530 millions d'euros pour des ordinateurs dans les prisons et seulement 70 millions d'euros pour les hôpitaux..! C'est quoi le problème de ce pays..? 


array(['"530 millions pour des ordi dans les prisons, seulement 70 pour les hôpitaux" : pourquoi ce visuel est trompeur 530 millions d\'euros pour des ordinateurs dans les prisons et seulement 70 millions d\'euros pour les hôpitaux',
       '"en douze mois, 39.000 retraités pauvres supplémentaires" en france ? ce chiffre date de 2013 le nombre de retraités pauvres a augmenté de 39.000 personnes sur les douze derniers mois',
       "70 milliards d'euros reversés à l'etat au titre des taxes sur les carburants ? un chiffre très surestimé l'etat touche 70 milliards d'euros grâce aux taxes sur le carburant",
       'video. la télévision « gratuite » en prison et « payante » dans les hôpitaux ? retour sur une intox « la télévision est payante dans les hôpitaux et gratuite dans les prisons »',
       "smic : une personne à temps complet perçoit-elle 1.450 euros ? pourquoi la déclaration d'elisabeth borne n'est pas juste – de facto – des clés pour mieux s'informer = smic : une personne à temps

In [24]:
print(df_posts_dev["full_text"].iloc[10])
idxs = df_posts_dev["pos_preds"].iloc[10]
df_posts_dev["preds_text"].iloc[10][idxs]

Le personnel qui travaille à l'hôpital souhaite que ce message soit diffusé à tous !!! ATTENTION ATTENTION A À partir de demain, ne quittez pas la maison et pour le pain, si vous pouvez congeler car le pire commence: la date d'incubation est respectée et de nombreux positifs vont commencer à sortir et beaucoup de gens peuvent le contracter ! il est donc très important de rester à la maison et ne pas interagir avec personne. Du 23 mars au 3 avril nous devons prendre soin de nous car nous serons au sommet du virus; la période d'incubation est de deux semaines, normalement durant ces deux semaines tous les infectés se déclarent, puis il y a deux semaines de calme et puis deux semaines où ils diminuent. NOUS SERONS EN STADE D'INFECTION MAXIMALE. Veuillez transmettre ce message à tous vos contacts. Le personnel qui travaille à l'hôpital souhaite que ce message soit diffusé à tous; #RESTEZCHEZVOUS Le personnel hospitalier souhaite que ce message soit diffusé à TOUS


array(['un calendrier prouvant que la livraison des vaccins pfizer est suspendue pendant la campagne présidentielle ? une "erreur de saisie", selon la dgs un calendrier prouve que la livraison des vaccins pfizer est suspendue',
       '"en stade d\'infection maximale" : ce message contient des informations infondées un pic épidémique à partir du 23 mars',
       "attention à ce message sur la période d’incubation de la covid-19 | coronavirus il faudrait éviter de sortir de chez soi jusqu'au 3 avril, car on atteint un « stade d'infection maximale »",
       'cristiano ronaldo et un hôpital au chili : une intox massivement reprise cristiano ronaldo a-t-il annoncé vouloir construire un hôpital pédiatrique au chili ?',
       "le pass sanitaire est-il nécessaire à l’hôpital en cas d’urgence ? c’est faux le pass sanitaire est nécessaire pour entrer à l'hôpital en cas d'urgence",
       "“pas d’aiguille” lors de la vaccination de kamala harris contre le covid-19 ? pas si vite... il n'y a pas

In [21]:
idxs = df_posts_dev["pos_preds"].iloc[0]
df_posts_dev["preds_text"].iloc[0][idxs]

array(['"530 millions pour des ordi dans les prisons, seulement 70 pour les hôpitaux" : pourquoi ce visuel est trompeur 530 millions d\'euros pour des ordinateurs dans les prisons et seulement 70 millions d\'euros pour les hôpitaux',
       '"en douze mois, 39.000 retraités pauvres supplémentaires" en france ? ce chiffre date de 2013 le nombre de retraités pauvres a augmenté de 39.000 personnes sur les douze derniers mois',
       "70 milliards d'euros reversés à l'etat au titre des taxes sur les carburants ? un chiffre très surestimé l'etat touche 70 milliards d'euros grâce aux taxes sur le carburant",
       'video. la télévision « gratuite » en prison et « payante » dans les hôpitaux ? retour sur une intox « la télévision est payante dans les hôpitaux et gratuite dans les prisons »',
       "smic : une personne à temps complet perçoit-elle 1.450 euros ? pourquoi la déclaration d'elisabeth borne n'est pas juste – de facto – des clés pour mieux s'informer = smic : une personne à temps

In [35]:
model = CrossEncoder("cross-encoder/stsb-distilroberta-base")


# We want to compute the similarity between the query sentence
query = "A man is eating pasta."

# With all sentences in the corpus
corpus = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
    "A woman is playing violin.",
    "Two men pushed carts through the woods.",
    "A man is riding a white horse on an enclosed ground.",
    "A monkey is playing drums.",
    "A cheetah is running behind its prey.",
]

# 1. We rank all sentences in the corpus for the query
ranks = model.rank(query, corpus)

# Print the scores
print("Query:", query)
for rank in ranks:
    print(f"{rank['score']:.2f}\t{corpus[rank['corpus_id']]}")

# 2. Alternatively, you can also manually compute the score between two sentences
sentence_combinations = [[query, sentence] for sentence in corpus]
scores = model.predict(sentence_combinations)

# Sort the scores in decreasing order to get the corpus indices
ranked_indices = np.argsort(scores)[::-1]
print("scores:", scores)
print("indices:", ranked_indices)

Query: A man is eating pasta.
0.67	A man is eating food.
0.34	A man is eating a piece of bread.
0.08	A man is riding a horse.
0.07	A man is riding a white horse on an enclosed ground.
0.01	The girl is carrying a baby.
0.01	Two men pushed carts through the woods.
0.01	A monkey is playing drums.
0.01	A woman is playing violin.
0.01	A cheetah is running behind its prey.
scores: [0.67323714 0.34102535 0.00542465 0.07569339 0.00525378 0.00536814
 0.06676241 0.00534824 0.00516718]
indices: [0 1 3 6 2 5 7 4 8]
